# Pollution impact factors for terrestrial acidification

Calculate impact factors for terrestrial acidification. IMPORTANT: If all cells are run you might run out of memory. To avoid this only run the ones you want to calculate e.g. the ones tagged with # NOx for calculating NOx impact factors. # COMMON cells should always be run.

Exiobase categories:
- Nox – combustion – air
- NH3 – combustion – air
- SOx – combustion – air

LC-Impact stressors
- Certain and all effects (PDF*yr/kg)
    - CF Nox
    - CF NH3
    - CF Sox

In [21]:
# import required libraries
import pymrio
import numpy as np
import pandas as pd
import pycountry as pyc
import json

# load arguments from json file
with open("../arguments.json", "r") as f:
    arguments = json.load(f)

In [23]:
# exiobase 2011 is used for calculating share of stressor for each region-product pair
exio3_11 = pymrio.parse_exiobase3(path=arguments["exio_11_path"])
# exiobase 2019 is used for impact factors
exio3_19 = pymrio.parse_exiobase3(path=arguments["exio_19_path"])

## Calculate DRorigin
DRorigin is a matrix which describes the amount of the driver of biodiversity loss (DR) that occurs in impact region i sector k and is driven by consumption in region j sector k.

To calculate the matrix we need to
1. Aggregate relevant drivers from exiobase together
2. Diagonalize the aggregated driver and re-calculate the IO system

In [24]:
# COMMON
# diagonalization

# make sure that L matrix is calculated 
if exio3_11.L is None:
    # try loading the L matrix from pickles/exio3_11_L.pickle
    try:
        exio3_11.L = pd.read_pickle("pickles/exio3_11_L.pickle")
        print("L matrix loaded from pickle")
    except FileNotFoundError:
        print("L matrix not found, calculating it from scratch.")
        print("Calculating A")
        exio3_11.A = pymrio.calc_A(exio3_11.Z, exio3_11.x)
        print("Calculating L")
        exio3_11.L = pymrio.calc_L(exio3_11.A)
        # save the L matrix to a pickle file
        pd.to_pickle(exio3_11.L, "pickles/exio3_11_L.pickle")
else:
    print("L already loaded")

Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()

L matrix loaded from pickle


/var/folders/bg/8zzl9cj57fnd569zy539k5h40000gn/T/ipykernel_12246/1974571270.py:21: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()


In [62]:
# NOx
# diagonalize and calculate
diag_nox = exio3_11.satellite.diag_stressor(("NH3 - combustion - air"))

# calculate S (direct emission multipliers)
if diag_nox.S is None:
    print("Calculating S for NOx")
    diag_nox.S = pymrio.calc_S(diag_nox.F, exio3_11.x)

diag_nox.D_cba, _, _, _ = pymrio.calc_accounts(diag_nox.S, exio3_11.L, Y_agg)

Calculating S for NOx


In [39]:
# NH3
# diagonaliza and calculate
diag_nh3 = exio3_11.satellite.diag_stressor(("NH3 - combustion - air"))

# calculate S (direct emission multipliers)
if diag_nh3.S is None:
    print("Calculating S for NH3")
    diag_nh3.S = pymrio.calc_S(diag_nh3.F, exio3_11.x)

diag_nh3.D_cba, _, _, _ = pymrio.calc_accounts(diag_nh3.S, exio3_11.L, Y_agg)

Calculating S for NH3


In [25]:
# SOx
# diagonalize and calculate
diag_sox = exio3_11.satellite.diag_stressor(("SOx - combustion - air"))

# calculate S (direct emission multipliers)
if diag_sox.S is None:
    print("Calculating S for SOx")
    diag_sox.S = pymrio.calc_S(diag_sox.F, exio3_11.x)

diag_sox.D_cba, _, _, _ = pymrio.calc_accounts(diag_sox.S, exio3_11.L, Y_agg)

Calculating S for SOx


## Calculate DR share
DR share is a new matrix that represents the share of the driver in the impact region i from the total amount of driver that is driven by consumption in region j sector k.

To calculate the matrix each column of DR origin is shared by the sum of that column.

In [63]:
# NOx
columns_nox = {}
for series_name, series in diag_nox.D_cba.items():
    series_sum = series.sum()
    columns_nox[series_name] = series / series_sum

dr_s_nox = pd.DataFrame(columns_nox)
dr_s_nox

AT  \
                                                            Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              4.720065e-09   
       Cereal grains nec                                  1.243251e-08   
       Vegetables, fruit, nuts                            1.819505e-09   
       Oil seeds                                          1.000225e-09   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       1.632992e-08   
       Recreational, cultural and sporting services (92)  3.281039e-08   
       Other services (93)                                6.057635e-08   
       Private households with employed persons (95)      2.306674e-09   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                                 Wheat   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              3.159039e-03   
       Cereal grains nec                                  9.638596e-08   
       Vegetables, fruit, nuts                            5.497940e-08   
       Oil seeds                                          1.762027e-08   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       1.467497e-08   
       Recreational, cultural and sporting services (92)  3.514500e-08   
       Other services (93)                                4.413505e-08   
       Private households with employed persons (95)      2.547866e-09   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                            \
                                                         Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  1.369177e-07   
       Cereal grains nec                                      5.283249e-03   
       Vegetables, fruit, nuts                                1.017523e-07   
       Oil seeds                                              2.756320e-08   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           1.530596e-08   
       Recreational, cultural and sporting services (92)      4.174231e-08   
       Other services (93)                                    4.731613e-08   
       Private households with employed persons (95)          2.852140e-09   
       Extra-territorial organizations and bodies             0.000000e+00   

                                                                                  \
                                                         Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        4.225382e-07   
       Cereal grains nec                                            6.227205e-07   
       Vegetables, fruit, nuts                                      1.629444e-02   
       Oil seeds                                                    6.862719e-08   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                 2.453872e-07   
       Recreational, cultural and sporting services (92)     

In [40]:
# NH3
columns_nh3 = {}
for series_name, series in diag_nh3.D_cba.items():
    series_sum = series.sum()
    columns_nh3[series_name] = series / series_sum

dr_s_nh3 = pd.DataFrame(columns_nh3)
dr_s_nh3

AT  \
                                                            Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              4.720065e-09   
       Cereal grains nec                                  1.243251e-08   
       Vegetables, fruit, nuts                            1.819505e-09   
       Oil seeds                                          1.000225e-09   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       1.632992e-08   
       Recreational, cultural and sporting services (92)  3.281039e-08   
       Other services (93)                                6.057635e-08   
       Private households with employed persons (95)      2.306674e-09   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                                 Wheat   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              3.159039e-03   
       Cereal grains nec                                  9.638596e-08   
       Vegetables, fruit, nuts                            5.497940e-08   
       Oil seeds                                          1.762027e-08   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       1.467497e-08   
       Recreational, cultural and sporting services (92)  3.514500e-08   
       Other services (93)                                4.413505e-08   
       Private households with employed persons (95)      2.547866e-09   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                            \
                                                         Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  1.369177e-07   
       Cereal grains nec                                      5.283249e-03   
       Vegetables, fruit, nuts                                1.017523e-07   
       Oil seeds                                              2.756320e-08   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           1.530596e-08   
       Recreational, cultural and sporting services (92)      4.174231e-08   
       Other services (93)                                    4.731613e-08   
       Private households with employed persons (95)          2.852140e-09   
       Extra-territorial organizations and bodies             0.000000e+00   

                                                                                  \
                                                         Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        4.225382e-07   
       Cereal grains nec                                            6.227205e-07   
       Vegetables, fruit, nuts                                      1.629444e-02   
       Oil seeds                                                    6.862719e-08   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                 2.453872e-07   
       Recreational, cultural and sporting services (92)     

In [26]:
# SOx
columns_sox = {}
for series_name, series in diag_sox.D_cba.items():
    series_sum = series.sum()
    columns_sox[series_name] = series / series_sum

dr_s_sox = pd.DataFrame(columns_sox)
dr_s_sox

AT  \
                                                            Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              1.143210e-07   
       Cereal grains nec                                  4.609239e-07   
       Vegetables, fruit, nuts                            6.876453e-09   
       Oil seeds                                          2.270163e-08   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       2.053488e-07   
       Recreational, cultural and sporting services (92)  1.670264e-06   
       Other services (93)                                4.006262e-06   
       Private households with employed persons (95)      4.150163e-08   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                                 Wheat   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              8.267790e-02   
       Cereal grains nec                                  3.861363e-06   
       Vegetables, fruit, nuts                            2.245265e-07   
       Oil seeds                                          4.321443e-07   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       1.994076e-07   
       Recreational, cultural and sporting services (92)  1.933276e-06   
       Other services (93)                                3.154106e-06   
       Private households with employed persons (95)      4.953497e-08   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                            \
                                                         Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  3.869707e-06   
       Cereal grains nec                                      2.285661e-01   
       Vegetables, fruit, nuts                                4.487411e-07   
       Oil seeds                                              7.300117e-07   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           2.245998e-07   
       Recreational, cultural and sporting services (92)      2.479653e-06   
       Other services (93)                                    3.651624e-06   
       Private households with employed persons (95)          5.988117e-08   
       Extra-territorial organizations and bodies             0.000000e+00   

                                                                                  \
                                                         Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        7.070079e-06   
       Cereal grains nec                                            1.594938e-05   
       Vegetables, fruit, nuts                                      4.254329e-02   
       Oil seeds                                                    1.076059e-06   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                 2.131770e-06   
       Recreational, cultural and sporting services (92)     

## Calculate DR unit
DR unit is a region harmonized version of DR share.

To calculate DR unit we need to:
1. Identify regions that are missing from exiobase (rest of the world regions) but are present in lc-impact.
2. Assume that the impacts are divided evenly across the rest of the world category
3. By that assumption we can add the missing countries to DR share as the impact of the row region where country belongs to divided by the number of countries in that region

In [64]:
# COMMON
# harmonize regions 

exio_regions = exio3_11.get_regions()
row_regions = {
    "WA": "Asia and pacific",
    "WE": "Europe",
    "WF": "Africa",
    "WM": "Middle east",
    "WL": "America"
}

# Function to convert country name to ISO Alpha-2 code
def get_country_code(name):
    # custom mappings for countries that pycountry does not recognize
    # these should cover all the countries in the LCI data if country has alpha-2 code
    # these mappings were extracted manually
    extra_mappings = {
        "Turkey": "TR",
        "Russia": "RU",
        "Bahamas, The": "BS",
        "Bonaire": "BQ",
        "Byelarus": "BY",
        "Brunei": "BN",
        "Cape Verde": "CV",
        "Cocos Islands": "CC",
        "Congo DRC": "CD",
        "China, Hong Kong Special Administrative Region": "HK",
        "Curacao": "CW",
        "Democratic Republic of the Congo": "CD",
        "Falkland Islands": "FK",
        "Falkland Islands (Islas Malvinas)": "FK",
        "Gambia, The": "GM",
        "Gaza Strip": "PS",
        "Heard Island & McDonald Islands": "HM",
        "Ivory Coast": "CI",
        "Macedonia": "MK",
        "The Former Yugoslav Republic of Macedonia": "MK",
        "Macau": "MO",
        "Man, Isle of": "IM",
        "Micronesia": "FM",
        "Myanmar (Burma)": "MM",
        "Netherlands Antilles": "AN",
        "Palestinian Territory": "PS",
        "Pacific Islands (Palau)": "PW",
        "Pitcairn Islands": "PN",
        "Reunion": "RE",
        "Saba": "BQ",
        "Saint Eustatius": "BQ",
        "Saint Helena": "SH",
        "Saint Martin": "MF",
        "Sint Maarten": "SX",
        "South Georgia and the South Sandwich Is": "GS",
        "South Georgia": "GS",
        "St. Helena": "SH",
        "Saint Barthelemy": "BL",
        "Saint Kitts and Nevis": "KN",
        "St. Kitts and Nevis": "KN",
        "St. Lucia": "LC",
        "St. Pierre and Miquelon": "PM",
        "Sao Tomo and Principe": "ST",
        "St. Vincent and the Grenadines": "VC",
        "Svalbard": "SJ",
        "Jan Mayen": "SJ",
        "Swaziland": "SZ",
        "US Virgin Islands": "VI",
        "Virgin Islands": "VG",
        "Western Samoa": "WS",
        "West Bank": "PS",
    }
    try:
        return pyc.countries.lookup(name).alpha_2
    except LookupError:
        try:
            return extra_mappings[name]
        except LookupError:
            print("Alpha-2 country code does not exist for ", name)
            return None

In [65]:
# COMMON
# load and prepare lc-impact data
lci = pd.read_excel(arguments["lc_impact_path"] + "/7-terrestrial acidification/CF_terrestrial_acidification.xlsx", 
                    sheet_name="CF per countries",
                    skiprows=0,
                    header=[0,1],
                    na_values=[' '])
lci.dropna(inplace=True) # for some reason there are empty rows in the data

# Flatten the multi-level columns and rename them
lci.columns = [' '.join(col).strip() for col in lci.columns]
lci.rename(columns={lci.columns[0]: "Country", lci.columns[1]: "CF Nox", lci.columns[2]: "CF NH3",lci.columns[3]: "CF Sox"}, inplace=True)

# Add country codes to the LCI data and drop rows without country codes
lci["Country_Code"] = lci["Country"].apply(get_country_code)
lci.dropna(subset=["Country_Code"], inplace=True)

lci

Alpha-2 country code does not exist for  Azores
Alpha-2 country code does not exist for  Canarias
Alpha-2 country code does not exist for  Madeira


,Country,CF Nox,CF NH3,CF Sox,Country_Code
0,Afghanistan,8.231737e-16,1.703934e-15,8.488037e-16,AF
1,Albania,5.948239e-14,4.414771e-14,1.318685e-13,AL
2,Algeria,4.098447e-15,5.971856e-15,3.989275e-15,DZ
3,Angola,8.554045e-16,5.537705e-16,5.879833e-16,AO
5,Argentina,1.106879e-15,3.149109e-16,1.272529e-15,AR
...,...,...,...,...,...
186,Venezuela,7.760938e-16,4.681585e-16,3.260127e-15,VE
187,Vietnam,9.745668e-17,2.501389e-16,2.083306e-16,VN
188,Yemen,7.643873e-14,6.878478e-14,5.564458e-15,YE
189,Zambia,7.486874e-16,6.257875e-16,1.405579e-16,ZM


In [66]:
# COMMON
# add regional averages for regions that are not in LCI data

def get_missing_from_lci(exio_regions, lci):
    """
    Get the regions that are in exiobase but not in lci data.
    """
    missing = []
    for region in exio_regions:
        if region not in lci["Country_Code"].tolist():
            missing.append(region)
    return missing

def augment_acid(lci_acidification):
    # taiwan and malta are missing from lc-impact
    cf_nox_asia = 2.68501157634878E-14
    cf_nh3_asia = 5.72697601775371E-15
    cf_sox_asia = 2.13657207308791E-14

    cf_nox_europe = 3.88566620512411E-14
    cf_nh3_europe = 1.209421972687E-14
    cf_sox_europe = 2.40053603985098E-14

    row_taiwan = pd.DataFrame({
        "Country": ["Taiwan"],
        "CF Nox": [cf_nox_asia],
        "CF NH3": [cf_nh3_asia],
        "CF Sox": [cf_sox_asia],
        "Country_Code": ["TW"],
    })
    row_malta = pd.DataFrame({
        "Country": ["Malta"],
        "CF Nox": [cf_nox_europe],
        "CF NH3": [cf_nh3_europe],
        "CF Sox": [cf_sox_europe],
        "Country_Code": ["MT"],
    })
    lci_acidification = pd.concat([lci_acidification, row_malta], ignore_index=True)
    lci_acidification = pd.concat([lci_acidification, row_taiwan], ignore_index=True)
    return lci_acidification

exio_regions_without_row = [region for region in exio_regions if region not in row_regions.keys()]
if len(get_missing_from_lci(exio_regions_without_row, lci)) > 0:
    print("Missing from LCI land stress:", get_missing_from_lci(exio_regions_without_row, lci))
    lci = augment_acid(lci)
    assert len(get_missing_from_lci(exio_regions_without_row, lci)) == 0, "There are still missing regions after augmentation"

Missing from LCI land stress: ['MT', 'TW']


In [67]:
# COMMON
# harmonize regions in LCI data

def get_row_regions(lci_country_codes, exio_country_codes):
    """
    Get the country codes from lci countries that don't exist in exiobase i.e. rest of the world countries.
    """
    row_regions = []
    for country in lci_country_codes:
        if country not in exio_country_codes:
            row_regions.append(country)
    
    # find duplicates in the list
    duplicates = []
    unique_regions = []
    seen_once = set()
    for item in row_regions:
        if item not in seen_once:
            unique_regions.append(item)
            seen_once.add(item)
        else:
            duplicates.append(item)
    if duplicates:
        print("Duplicates found in row regions:", duplicates)
    return unique_regions

row_countries = get_row_regions(lci["Country_Code"].tolist(), exio_regions)
print("Row regions:", row_countries)

# Load region mappings from arguments
row_eu_countries = arguments["row_region_mappings"]["row_eu"]
row_asia_pacific_countries = arguments["row_region_mappings"]["row_asia_pacific"]
row_african_countries = arguments["row_region_mappings"]["row_africa"]
row_american_countries = arguments["row_region_mappings"]["row_america"]
row_middle_eastern_countries = arguments["row_region_mappings"]["row_middle_east"]

Row regions: ['AF', 'AL', 'DZ', 'AO', 'AR', 'AM', 'AZ', 'BS', 'BD', 'BY', 'BZ', 'BJ', 'BT', 'BO', 'BA', 'BW', 'BN', 'BF', 'BI', 'KH', 'CM', 'KY', 'CF', 'TD', 'CL', 'CO', 'KM', 'CG', 'CD', 'CR', 'CI', 'CU', 'DJ', 'DO', 'EC', 'EG', 'SV', 'GQ', 'ER', 'ET', 'FK', 'GF', 'GA', 'GM', 'GE', 'GH', 'GP', 'GT', 'GN', 'GW', 'GY', 'HT', 'HN', 'IS', 'IR', 'IQ', 'IL', 'JM', 'JE', 'JO', 'KZ', 'KE', 'KW', 'KG', 'LA', 'LB', 'LS', 'LR', 'LY', 'MG', 'MW', 'MY', 'ML', 'MR', 'MU', 'MD', 'MN', 'ME', 'MA', 'MZ', 'MM', 'NA', 'NP', 'NZ', 'NI', 'NE', 'NG', 'KP', 'OM', 'PK', 'PS', 'PA', 'PG', 'PY', 'PE', 'PH', 'PR', 'QA', 'RE', 'RW', 'VC', 'SA', 'SN', 'RS', 'SL', 'SO', 'SS', 'LK', 'SD', 'SR', 'SZ', 'SY', 'TJ', 'TZ', 'TH', 'MK', 'TL', 'TG', 'TT', 'TN', 'TM', 'UG', 'UA', 'AE', 'UY', 'VI', 'UZ', 'VE', 'VN', 'YE', 'ZM', 'ZW']


In [ ]:
# NOx
# augment dr_s to create dr_u

# new regions are calculated by dividing their corresponding row region by the number of countries in the row region
# for example, row region Argentina is sub-matrix WA divided by the number of countries in row region WA
wl_nox = dr_s_nox.loc["WL"].copy()
wl_nox = wl_nox / len(row_american_countries)

we_nox = dr_s_nox.loc["WE"].copy()
we_nox = we_nox / len(row_eu_countries)

wa_nox = dr_s_nox.loc["WA"].copy()
wa_nox = wa_nox / len(row_asia_pacific_countries)

wf_nox = dr_s_nox.loc["WF"].copy()
wf_nox = wf_nox / len(row_african_countries)

wm_nox = dr_s_nox.loc["WM"].copy()
wm_nox = wm_nox / len(row_middle_eastern_countries)

dr_u_nox = dr_s_nox.copy()
dr_u_nox = dr_u_nox.drop(index=row_regions.keys(), level='region')

# build a mapping of country codes to region dataframes
country_to_region_nox = {}
for region in row_countries:
    if region in row_eu_countries:
        country_to_region_nox[region] = we_nox
    elif region in row_asia_pacific_countries:
        country_to_region_nox[region] = wa_nox
    elif region in row_african_countries:
        country_to_region_nox[region] = wf_nox
    elif region in row_american_countries:
        country_to_region_nox[region] = wl_nox
    elif region in row_middle_eastern_countries:
        country_to_region_nox[region] = wm_nox
    else:
        raise ValueError(f"Unknown region: {region}")

# add all new regions to dr_u
all_indices = []
all_data = []
for region in row_countries:
    region_data = country_to_region_nox[region].copy()
    idx = pd.MultiIndex.from_product([[region],region_data.index], names=['region', 'sector'])
    all_indices.append(idx)
    all_data.append(region_data)

combined_idx_nox = pd.MultiIndex.from_tuples(
    [idx for subidx in all_indices for idx in subidx]
)

combined_data_nox = pd.concat(all_data)
combined_data_nox.index = combined_idx_nox

dr_u_nox = pd.concat([dr_u_nox, combined_data_nox])
# drop row region columns
dr_u_nox = dr_u_nox.drop(columns=row_regions.keys(), axis=1, level=0)
dr_u_nox

AT  \
                                                        Paddy rice   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              4.720065e-09   
   Cereal grains nec                                  1.243251e-08   
   Vegetables, fruit, nuts                            1.819505e-09   
   Oil seeds                                          1.000225e-09   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       9.247303e-11   
   Recreational, cultural and sporting services (92)  3.085748e-10   
   Other services (93)                                3.089864e-10   
   Private households with employed persons (95)      6.401889e-11   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                    \
                                                             Wheat   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              3.159039e-03   
   Cereal grains nec                                  9.638596e-08   
   Vegetables, fruit, nuts                            5.497940e-08   
   Oil seeds                                          1.762027e-08   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       2.347154e-10   
   Recreational, cultural and sporting services (92)  1.087058e-09   
   Other services (93)                                6.150415e-10   
   Private households with employed persons (95)      1.596221e-10   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  1.369177e-07   
   Cereal grains nec                                      5.283249e-03   
   Vegetables, fruit, nuts                                1.017523e-07   
   Oil seeds                                              2.756320e-08   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)           2.512129e-10   
   Recreational, cultural and sporting services (92)      1.333865e-09   
   Other services (93)                                    6.616163e-10   
   Private households with employed persons (95)          1.417562e-10   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        4.225382e-07   
   Cereal grains nec                                            6.227205e-07   
   Vegetables, fruit, nuts                                      1.629444e-02   
   Oil seeds                                                    6.862719e-08   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                 2.340757e-09   
   Recreational, cultural and sporting services (92)            2.747844e-09   
   Other services (93)                                          1.792450e-09   
   Private households with employed persons (95)                3.712493e-10   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                    

In [69]:
# NH3
# augment dr_s to create dr_u

# new regions are calculated by dividing their corresponding row region by the number of countries in the row region
# for example, row region Argentina is sub-matrix WA divided by the number of countries in row region WA
wl_nh3 = dr_s_nh3.loc["WL"].copy()
wl_nh3 = wl_nh3 / len(row_american_countries)

we_nh3 = dr_s_nh3.loc["WE"].copy()
we_nh3 = we_nh3 / len(row_eu_countries)

wa_nh3 = dr_s_nh3.loc["WA"].copy()
wa_nh3 = wa_nh3 / len(row_asia_pacific_countries)

wf_nh3 = dr_s_nh3.loc["WF"].copy()
wf_nh3 = wf_nh3 / len(row_african_countries)

wm_nh3 = dr_s_nh3.loc["WM"].copy()
wm_nh3 = wm_nh3 / len(row_middle_eastern_countries)

dr_u_nh3 = dr_s_nh3.copy()
dr_u_nh3 = dr_u_nh3.drop(index=row_regions.keys(), level='region')

# build a mapping of country codes to region dataframes
country_to_region = {}
for region in row_countries:
    if region in row_eu_countries:
        country_to_region[region] = we_nh3
    elif region in row_asia_pacific_countries:
        country_to_region[region] = wa_nh3
    elif region in row_african_countries:
        country_to_region[region] = wf_nh3
    elif region in row_american_countries:
        country_to_region[region] = wl_nh3
    elif region in row_middle_eastern_countries:
        country_to_region[region] = wm_nh3
    else:
        raise ValueError(f"Unknown region: {region}")

# add all new regions to dr_u
all_indices = []
all_data = []
for region in row_countries:
    region_data = country_to_region[region].copy()
    idx = pd.MultiIndex.from_product([[region],region_data.index], names=['region', 'sector'])
    all_indices.append(idx)
    all_data.append(region_data)

combined_idx = pd.MultiIndex.from_tuples(
    [idx for subidx in all_indices for idx in subidx]
)

combined_data = pd.concat(all_data)
combined_data.index = combined_idx

dr_u_nh3 = pd.concat([dr_u_nh3, combined_data])
# drop row region columns
dr_u_nh3 = dr_u_nh3.drop(columns=row_regions.keys(), axis=1, level=0)
dr_u_nh3

AT  \
                                                        Paddy rice   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              4.720065e-09   
   Cereal grains nec                                  1.243251e-08   
   Vegetables, fruit, nuts                            1.819505e-09   
   Oil seeds                                          1.000225e-09   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       9.247303e-11   
   Recreational, cultural and sporting services (92)  3.085748e-10   
   Other services (93)                                3.089864e-10   
   Private households with employed persons (95)      6.401889e-11   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                    \
                                                             Wheat   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              3.159039e-03   
   Cereal grains nec                                  9.638596e-08   
   Vegetables, fruit, nuts                            5.497940e-08   
   Oil seeds                                          1.762027e-08   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       2.347154e-10   
   Recreational, cultural and sporting services (92)  1.087058e-09   
   Other services (93)                                6.150415e-10   
   Private households with employed persons (95)      1.596221e-10   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  1.369177e-07   
   Cereal grains nec                                      5.283249e-03   
   Vegetables, fruit, nuts                                1.017523e-07   
   Oil seeds                                              2.756320e-08   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)           2.512129e-10   
   Recreational, cultural and sporting services (92)      1.333865e-09   
   Other services (93)                                    6.616163e-10   
   Private households with employed persons (95)          1.417562e-10   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        4.225382e-07   
   Cereal grains nec                                            6.227205e-07   
   Vegetables, fruit, nuts                                      1.629444e-02   
   Oil seeds                                                    6.862719e-08   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                 2.340757e-09   
   Recreational, cultural and sporting services (92)            2.747844e-09   
   Other services (93)                                          1.792450e-09   
   Private households with employed persons (95)                3.712493e-10   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                    

In [ ]:
# SOx
# augment dr_s to create dr_u

# new regions are calculated by dividing their corresponding row region by the number of countries in the row region
# for example, row region Argentina is sub-matrix WA divided by the number of countries in row region WA
wl_sox = dr_s_sox.loc["WL"].copy()
wl_sox = wl_sox / len(row_american_countries)

we_sox = dr_s_sox.loc["WE"].copy()
we_sox = we_sox / len(row_eu_countries)

wa_sox = dr_s_sox.loc["WA"].copy()
wa_sox = wa_sox / len(row_asia_pacific_countries)

wf_sox = dr_s_sox.loc["WF"].copy()
wf_sox = wf_sox / len(row_african_countries)

wm_sox = dr_s_sox.loc["WM"].copy()
wm_sox = wm_sox / len(row_middle_eastern_countries)

dr_u_sox = dr_s_sox.copy()
dr_u_sox = dr_u_sox.drop(index=row_regions.keys(), level='region')

# build a mapping of country codes to region dataframes
country_to_region = {}
for region in row_countries:
    if region in row_eu_countries:
        country_to_region[region] = we_sox
    elif region in row_asia_pacific_countries:
        country_to_region[region] = wa_sox
    elif region in row_african_countries:
        country_to_region[region] = wf_sox
    elif region in row_american_countries:
        country_to_region[region] = wl_sox
    elif region in row_middle_eastern_countries:
        country_to_region[region] = wm_sox
    else:
        raise ValueError(f"Unknown region: {region}")

# add all new regions to dr_u
all_indices = []
all_data = []
for region in row_countries:
    region_data = country_to_region[region].copy()
    idx = pd.MultiIndex.from_product([[region],region_data.index], names=['region', 'sector'])
    all_indices.append(idx)
    all_data.append(region_data)

combined_idx = pd.MultiIndex.from_tuples(
    [idx for subidx in all_indices for idx in subidx]
)

combined_data = pd.concat(all_data)
combined_data.index = combined_idx

dr_u_sox = pd.concat([dr_u_sox, combined_data])
# drop row region columns
dr_u_sox = dr_u_sox.drop(columns=row_regions.keys(), axis=1, level=0)
dr_u_sox

AT  \
                                                        Paddy rice   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              1.143210e-07   
   Cereal grains nec                                  4.609239e-07   
   Vegetables, fruit, nuts                            6.876453e-09   
   Oil seeds                                          2.270163e-08   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       8.579651e-10   
   Recreational, cultural and sporting services (92)  3.886656e-09   
   Other services (93)                                5.335510e-09   
   Private households with employed persons (95)      3.195917e-10   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                    \
                                                             Wheat   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              8.267790e-02   
   Cereal grains nec                                  3.861363e-06   
   Vegetables, fruit, nuts                            2.245265e-07   
   Oil seeds                                          4.321443e-07   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       2.353166e-09   
   Recreational, cultural and sporting services (92)  1.479533e-08   
   Other services (93)                                1.147618e-08   
   Private households with employed persons (95)      8.610666e-10   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  3.869707e-06   
   Cereal grains nec                                      2.285661e-01   
   Vegetables, fruit, nuts                                4.487411e-07   
   Oil seeds                                              7.300117e-07   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)           2.719800e-09   
   Recreational, cultural and sporting services (92)      1.960505e-08   
   Other services (93)                                    1.333163e-08   
   Private households with employed persons (95)          8.257906e-10   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        7.070079e-06   
   Cereal grains nec                                            1.594938e-05   
   Vegetables, fruit, nuts                                      4.254329e-02   
   Oil seeds                                                    1.076059e-06   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                 1.500346e-08   
   Recreational, cultural and sporting services (92)            2.391047e-08   
   Other services (93)                                          2.138277e-08   
   Private households with employed persons (95)                1.280364e-09   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                    

## Calculate DR factor
Calculate the impact factors of the driver for each impact region i driven by consumption in region j product sector k.

These impact factors tell how

To calculate DR factor:
1. Calculate the monetary impact factor impact/€ from 2019 exiobase data
2. Multiply each column of DR unit with the impact factors of consumption region j in product sector k. The resulting matrix represents the distribution of impacts of 1€ consumption in each consumption region.

In [70]:
# NOx

# use 2019 impact factors for calculating dr_f
# calculate dr_f - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
dr_f_nox = dr_u_nox.copy()
satellite_cleaned = exio3_19.satellite.M.drop(columns=row_regions.keys(), axis=1, level=0)
total = satellite_cleaned.loc["NOx - combustion - air"]
scalars = total.to_numpy() # multipliers for each column
print(len(scalars))
print(dr_u_nox.shape)

# multiply each column of dr_u by the respective column value 
dr_f_nox = dr_f_nox * scalars
dr_f_nox

8800
(35200, 8800)


AT                \
                                                     Paddy rice         Wheat   
AT Paddy rice                                               0.0  0.000000e+00   
   Wheat                                                    0.0  1.721285e+01   
   Cereal grains nec                                        0.0  5.251839e-04   
   Vegetables, fruit, nuts                                  0.0  2.995695e-04   
   Oil seeds                                                0.0  9.600864e-05   
...                                                         ...           ...   
ZW Membership organisation services n.e.c. (91)             0.0  1.278908e-06   
   Recreational, cultural and sporting services (92)        0.0  5.923117e-06   
   Other services (93)                                      0.0  3.351213e-06   
   Private households with employed persons (95)            0.0  8.697423e-07   
   Extra-territorial organizations and bodies               0.0  0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  4.053511e-04   
   Cereal grains nec                                      1.564130e+01   
   Vegetables, fruit, nuts                                3.012423e-04   
   Oil seeds                                              8.160212e-05   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)           7.437273e-07   
   Recreational, cultural and sporting services (92)      3.948968e-06   
   Other services (93)                                    1.958745e-06   
   Private households with employed persons (95)          4.196758e-07   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        2.904805e-04   
   Cereal grains nec                                            4.280991e-04   
   Vegetables, fruit, nuts                                      1.120187e+01   
   Oil seeds                                                    4.717885e-05   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                 1.609191e-06   
   Recreational, cultural and sporting services (92)            1.889049e-06   
   Other services (93)                                          1.232248e-06   
   Private households with employed persons (95)                2.552212e-07   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              7.182924e-05   
   Cereal grains nec                                  7.517291e-05   
   Vegetables, fruit, nuts                            4.915163e-05   
   Oil seeds                                          2.551839e-01   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       6.637771e-07   
   Recreational, cultural and sporting services (92)  2.213879e-06   
   Other services (93)                                1.638415e-06   
   Private households with employed persons (95)      5.671739e-07   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                             \
             

In [57]:
# NH3

# use 2019 impact factors for calculating dr_f
# calculate dr_f - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
dr_f_nh3 = dr_u_nh3.copy()
satellite_cleaned = exio3_19.satellite.M.drop(columns=row_regions.keys(), axis=1, level=0)
total = satellite_cleaned.loc["NH3 - combustion - air"]
scalars = total.to_numpy() # multipliers for each column
print(len(scalars))
print(dr_u_nh3.shape)

# multiply each column of dr_u by the respective column value 
dr_f_nh3 = dr_f_nh3 * scalars
dr_f_nh3

8800
(35200, 8800)


AT                \
                                                     Paddy rice         Wheat   
AT Paddy rice                                               0.0  0.000000e+00   
   Wheat                                                    0.0  1.613102e-01   
   Cereal grains nec                                        0.0  4.921761e-06   
   Vegetables, fruit, nuts                                  0.0  2.807416e-06   
   Oil seeds                                                0.0  8.997449e-07   
...                                                         ...           ...   
ZW Membership organisation services n.e.c. (91)             0.0  1.198528e-08   
   Recreational, cultural and sporting services (92)        0.0  5.550849e-08   
   Other services (93)                                      0.0  3.140589e-08   
   Private households with employed persons (95)            0.0  8.150790e-09   
   Extra-territorial organizations and bodies               0.0  0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  7.257714e-06   
   Cereal grains nec                                      2.800537e-01   
   Vegetables, fruit, nuts                                5.393670e-06   
   Oil seeds                                              1.461066e-06   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)           1.331626e-08   
   Recreational, cultural and sporting services (92)      7.070532e-08   
   Other services (93)                                    3.507086e-08   
   Private households with employed persons (95)          7.514193e-09   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        1.110172e-05   
   Cereal grains nec                                            1.636129e-05   
   Vegetables, fruit, nuts                                      4.281183e-01   
   Oil seeds                                                    1.803103e-06   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                 6.150079e-08   
   Recreational, cultural and sporting services (92)            7.219655e-08   
   Other services (93)                                          4.709463e-08   
   Private households with employed persons (95)                9.754163e-09   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              6.682478e-07   
   Cereal grains nec                                  6.993550e-07   
   Vegetables, fruit, nuts                            4.572717e-07   
   Oil seeds                                          2.374048e-03   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       6.175307e-09   
   Recreational, cultural and sporting services (92)  2.059635e-08   
   Other services (93)                                1.524264e-08   
   Private households with employed persons (95)      5.276581e-09   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                             \
             

In [33]:
# SOx

# use 2019 impact factors for calculating dr_f
# calculate dr_f - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
dr_f_sox = dr_u_sox.copy()
satellite_cleaned = exio3_19.satellite.M.drop(columns=row_regions.keys(), axis=1, level=0)
total = satellite_cleaned.loc["SOx - combustion - air"]
scalars = total.to_numpy() # multipliers for each column
print(len(scalars))
print(dr_u_sox.shape)

# multiply each column of dr_u by the respective column value 
dr_f_sox = dr_f_sox * scalars
dr_f_sox

8800
(35200, 8800)


AT                \
                                                     Paddy rice         Wheat   
AT Paddy rice                                               0.0  0.000000e+00   
   Wheat                                                    0.0  3.817790e+01   
   Cereal grains nec                                        0.0  1.783049e-03   
   Vegetables, fruit, nuts                                  0.0  1.036789e-04   
   Oil seeds                                                0.0  1.995498e-04   
...                                                         ...           ...   
ZW Membership organisation services n.e.c. (91)             0.0  1.086614e-06   
   Recreational, cultural and sporting services (92)        0.0  6.831993e-06   
   Other services (93)                                      0.0  5.299319e-06   
   Private households with employed persons (95)            0.0  3.976119e-07   
   Extra-territorial organizations and bodies               0.0  0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  1.664988e-03   
   Cereal grains nec                                      9.834331e+01   
   Vegetables, fruit, nuts                                1.930762e-04   
   Oil seeds                                              3.140962e-04   
...                                                                ...   
ZW Membership organisation services n.e.c. (91)           1.170227e-06   
   Recreational, cultural and sporting services (92)      8.435307e-06   
   Other services (93)                                    5.736093e-06   
   Private households with employed persons (95)          3.553063e-07   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        1.109106e-03   
   Cereal grains nec                                            2.502031e-03   
   Vegetables, fruit, nuts                                      6.673904e+00   
   Oil seeds                                                    1.688049e-04   
...                                                                      ...   
ZW Membership organisation services n.e.c. (91)                 2.353641e-06   
   Recreational, cultural and sporting services (92)            3.750913e-06   
   Other services (93)                                          3.354385e-06   
   Private households with employed persons (95)                2.008548e-07   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              2.028086e-04   
   Cereal grains nec                                  3.248920e-04   
   Vegetables, fruit, nuts                            2.165491e-05   
   Oil seeds                                          6.751809e-01   
...                                                            ...   
ZW Membership organisation services n.e.c. (91)       7.179333e-07   
   Recreational, cultural and sporting services (92)  3.250700e-06   
   Other services (93)                                3.298130e-06   
   Private households with employed persons (95)      3.300737e-07   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                             \
             

## Calculate BDe factors (PDF/€ for each consumption region and product sector)

1. Multiply each cell of dr_u with the CF (characterisation factor from lc-impact) of the impact region to get PDF values for every entry
2. Sum up columns to get the total PDF/€ for the consumption region j product sector k

In [71]:
# NOx
lci_copy_nox = lci.copy()
lci_copy_nox.set_index("Country_Code", inplace=True)

# Ensure lci index is unique before reindexing
lci_copy_nox = lci_copy_nox[~lci_copy_nox.index.duplicated(keep='first')]

# sort rows on lci in same order as dr_f_nox.index.sortlevel
lci_reindexed_nox = lci_copy_nox.reindex(dr_f_nox.index.get_level_values(0).unique())

# build array from the relevant lci stressor
cf_nox = lci_reindexed_nox["CF Nox"].to_numpy()
cf_nox = np.repeat(cf_nox, 200)  # 1D array of length 200 * number of regions in lci
cf_nox = np.tile(cf_nox, (dr_f_nox.shape[1], 1)).T

pdf_nox = dr_f_nox * cf_nox
pdf_total_nox = pdf_nox.sum()

In [ ]:
# NH3
lci_copy_nh3 = lci.copy()
lci_copy_nh3.set_index("Country_Code", inplace=True)

# Ensure lci index is unique before reindexing
lci_copy_nh3 = lci_copy_nh3[~lci_copy_nh3.index.duplicated(keep='first')]

# sort rows on lci in same order as dr_f_nh3.index.sortlevel
lci_reindexed_nh3 = lci_copy_nh3.reindex(dr_f_nh3.index.get_level_values(0).unique())

# build array from the relevant lci stressor
cf_nh3 = lci_reindexed_nh3["CF NH3"].to_numpy()
cf_nh3 = np.repeat(cf_nh3, 200)  # 1D array of length 200 * number of regions in lci
cf_nh3 = np.tile(cf_nh3, (dr_f_nh3.shape[1], 1)).T

pdf_nh3 = dr_f_nh3 * cf_nh3
pdf_total_nh3 = pdf_nh3.sum()

In [ ]:
# SOx
lci_copy_sox = lci.copy()
lci_copy_sox.set_index("Country_Code", inplace=True)

# Ensure lci index is unique before reindexing
lci_copy_sox = lci_copy_sox[~lci_copy_sox.index.duplicated(keep='first')]

# sort rows on lci in same order as dr_f_sox.index.sortlevel
lci_reindexed_sox = lci_copy_sox.reindex(dr_f_sox.index.get_level_values(0).unique())

# build array from the relevant lci stressor
cf_sox = lci_reindexed_sox["CF Sox"].to_numpy()
cf_sox = np.repeat(cf_sox, 200)  # 1D array of length 200 * number of regions in lci
cf_sox = np.tile(cf_sox, (dr_f_sox.shape[1], 1)).T

pdf_sox = dr_f_sox * cf_sox
pdf_total_sox = pdf_sox.sum()

In [ ]:

# NOx
pd.DataFrame(pdf_total_nox).to_csv("csv/pdf-pollution-acidification-nox.csv", index=True)

In [ ]:

# NH3
pd.DataFrame(pdf_total_nh3).to_csv("csv/pdf-pollution-acidification-nh3.csv", index=True)

In [ ]:
# SOx
pd.DataFrame(pdf_total_sox).to_csv("csv/pdf-pollution-acidification-sox.csv", index=True)